# Setting the OpenAI Key as an Environment Variable

In [1]:
%load_ext dotenv
%dotenv
%load_ext mypy_ipython

# Importing Relevant Classes and functions

In [5]:
# Library Import
from langgraph.graph import START, END, StateGraph
from typing_extensions import TypedDict
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.runnables import Runnable
from langchain_core.messages import HumanMessage, BaseMessage
from collections.abc import Sequence

# Define the State
class State(TypedDict):
    message: list[BaseMessage]

# Define Chat Model
chat = ChatOpenAI(model='gpt-4',
                  seed= 25,
                  max_completion_tokens= 100,
                  temperature=0
)

# Define Node
def chatbot(state: State) -> State:
    print(f"\n---------> Entering Chatbot")
    AI_message = chat.invoke(state['message'])
    AI_message.pretty_print()

    return State(message = [AI_message])

# Initial State
state = State(
    message = [
        HumanMessage("Could you let me know what is the best time to visit kolkata ?")
    ]
)
state['message'][0].pretty_print()

# Test node directly
# new_state = chatbot(state)

# Define Graph (typed)
graph : StateGraph[State] = StateGraph(State)
graph.add_node('chatbot', chatbot)
graph.add_edge(START, 'chatbot')
graph.add_edge('chatbot', END)

graph_compiled = graph.compile()

# Invoke Graph
final_state = graph_compiled.invoke(state)

================================ Human Message =================================

Could you let me know what is the best time to visit kolkata ?

---------> Entering Chatbot
================================== Ai Message ==================================

The best time to visit Kolkata, India is during the cooler months from October to March. This period sees a lot of cultural activities like music festivals, art exhibitions, and theater. The famous Durga Puja, which is a highlight in Kolkata, is usually celebrated around October. The weather during these months is pleasant and comfortable for sightseeing.
